### Message Passing for REFLACX graph datapoints

In [ ]:
import torch
import dgl
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.device_count() > 0:
    torch.set_default_device('cuda')
ds = dgl.data.CSVDataset('datasets/reflacx_densnet225_iou')

In [ ]:
g, labels = ds[36]

In [ ]:
if torch.cuda.device_count() > 0:
    g = g.to(torch.device('cuda:0'))
    labels = labels.to(torch.device('cuda:0'))
g, labels, g.nodes().device, labels.device

In [ ]:
options = {
    'node_color': 'black',
    'node_size': 20,
    'width': 1,
}
G = dgl.to_networkx(g.cpu())
plt.figure(figsize=[15,7])
nx.draw(G, **options)

In [ ]:
def get_node(graph, i):
    assert 0 <= i < len(graph.nodes())
    result = {}
    for k in graph.ndata.keys():
        result[k] = graph.ndata[k][i]
    return result

In [ ]:
def get_edge(graph, i, j):
    try:
        e_i = graph.edge_ids(i, j)
    except dgl.DGLError:
        return 0.0
    return graph.edata['weight'][e_i] # TODO review field name str

In [ ]:
node = lambda i: get_node(g, i)
edge = lambda i, j: get_edge(g, i ,j)

In [ ]:
node(0), node(1), edge(0, 1)